In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from dblib import mssql
print('done')

In [16]:
#load data
#mv_graph_plat_cam_net.sql
sql = "select ifca, intertidalfid, season,species,n_sum,n_count from v_all_aggregate order by ifca, intertidalfid, season, species, n_sum desc"
with mssql.Conn('mmo', 's30', user='sa', pw='GGM290471') as cnn:
    df = pd.read_sql(sql, cnn)
df['n_count_ln'] = np.log( df['n_count'])
list(df)    

['ifca',
 'intertidalfid',
 'season',
 'species',
 'n_sum',
 'n_count',
 'n_count_ln']

In [17]:
#BY SPECIES/SEASON

In [ ]:
df['rank'] = df.groupby(['ifca', 'season', 'species'])['n_count_ln'].transform(lambda x: pd.cut(x, 3, labels=range(1,4), duplicates='drop'))
df.to_csv('C:/Users/Graham Monkman/OneDrive/Documents/MMOMapping/jupyter/outputs/mmo_species_rank.csv')
print('Done')

In [ ]:
#NWW OVERALL

In [18]:
sql = "select ifca, intertidalfid, survey_poly_name, survey_poly_ifca, count(*) as post_n from v_all group by ifca, intertidalfid, survey_poly_name, survey_poly_ifca order by ifca, intertidalfid"
with mssql.Conn('mmo', 's30', user='sa', pw='GGM290471') as cnn:
    df_all = pd.read_sql(sql, cnn)
    
df_all['n_count_log'] = np.log( df_all['post_n'])
list(df_all)

Data loaded


In [ ]:
#OVERALL

In [7]:
df_al

,ifca,intertidalfid,survey_poly_name,survey_poly_ifca,post_n
0,cornwall,15,None,None,16
1,cornwall,23,None,None,31
2,cornwall,25,None,None,8
3,cornwall,29,None,None,126
4,cornwall,30,None,None,187
5,cornwall,34,None,None,3
6,cornwall,75,None,None,36
7,cornwall,78,None,None,13
8,cornwall,108,None,None,61
9,cornwall,109,None,None,1


In [19]:
df_all['rank_total'] = df_all.groupby(['ifca'])['post_n'].transform(lambda x: pd.qcut(x, 3, labels=range(1,4), duplicates='drop'))

In [21]:
df_all.to_csv('C:/Users/Graham Monkman/OneDrive/Documents/MMOMapping/jupyter/outputs/mmo_overall_rank.csv')